In [1]:
import sys
sys.path.append("..")
import tools
import tensorflow as tf
import numpy as np
import json
from matplotlib import pyplot as plt
%load_ext autoreload
%autoreload 2

2024-04-14 02:32:13.318505: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 02:32:13.318559: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 02:32:13.319488: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-14 02:32:13.326076: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-14 02:32:17.428532: W tensorflow/compiler/tf2

In [11]:
repo = "/epyc/ssd/users/kmrakovc/DATA/rc2_subset/SMALL_HSC/"
output_coll = "u/kmrakovc/single_frame_injection_05"
val_index = tools.data.convert_butler_tfrecords (repo, output_coll, shape=(128,128), 
                                     filename_train="/astro/users/kmrakovc/Projects/LSST_Streak_Detection/DATA/train5.tfrecord",
                                     filename_test="/astro/users/kmrakovc/Projects/LSST_Streak_Detection/DATA/test5.tfrecord")

 234 / 234

In [4]:
val_index = np.array(val_index)
val_index.sort()
with open("/astro/users/kmrakovc/Projects/LSST_Streak_Detection/DATA/val_index1.npy", 'wb') as f:
    np.save(f, val_index)
print (val_index)

[  9  10  15  16  18  19  24  25  30  45  55  60  66  69  73  82  84  86
  93  95  97 108 109 112 117 124 125 127 139 144 146 147 148 155 156 158
 161 162 164 168 170 177 180 183 196 199 201 203 206 208 213 218 220 224
 225 227 229 231]


In [7]:
with open("/astro/users/kmrakovc/Projects/LSST_Streak_Detection/DATA/val_index2.npy", 'rb') as f:
    print (np.load(f))
print (val_index)

[  9  10  15  16  18  19  24  25  30  45  55  60  66  69  73  82  84  86
  93  95  97 108 109 112 117 124 125 127 139 144 146 147 148 155 156 158
 161 162 164 168 170 177 180 183 196 199 201 203 206 208 213 218 220 224
 225 227 229 231]
[  9  10  15  16  18  19  24  25  30  45  55  60  66  69  73  82  84  86
  93  95  97 108 109 112 117 124 125 127 139 144 146 147 148 155 156 158
 161 162 164 168 170 177 180 183 196 199 201 203 206 208 213 218 220 224
 225 227 229 231]


In [3]:
dataset_train = tf.data.TFRecordDataset(["/astro/users/kmrakovc/Projects/LSST_Streak_Detection/DATA/train.tfrecord"])
tfrecord_shape = tools.model.get_shape_of_quadratic_image_tfrecord(dataset_train)
dataset_train = dataset_train.map(tools.model.parse_function(img_shape=tfrecord_shape, test=False))
#dataset_train = dataset_train.batch(128).prefetch(2)
dataset_val = tf.data.TFRecordDataset(["/astro/users/kmrakovc/Projects/LSST_Streak_Detection/DATA/test.tfrecord"])
dataset_val = dataset_val.map(tools.model.parse_function(img_shape=tfrecord_shape, test=False))
#dataset_val = dataset_val.batch(128).prefetch(2)

2024-03-26 13:27:42.805065: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [380]:
print ("Train dataset:")
for i,j in enumerate(dataset_train):
    print ("\r",i, end="")
print ("\nTest dataset:")
for i,j in enumerate(dataset_val):
    print ("\r",i, end="")

Train dataset:
 92927
Test dataset:
 30623

In [12]:
dataset_train_merged = tools.data.merge (dataset_train, (4176, 2048))
dataset_val_merged = tools.data.merge (dataset_val, (4176, 2048))

In [13]:
def dataset_to_numpy(dataset):
    for i, a in enumerate(dataset):
        s = a.shape
    array = np.empty((i+1)+s)
    for i, a in enumerate(dataset):
        array[i] = a
    return array
test = dataset_to_numpy(dataset_train_merged)
val = dataset_to_numpy(dataset_val_merged)

In [15]:
test.shape, val.shape

((176, 4176, 2048), (58, 4176, 2048))

In [17]:
tools.data.get_asteroid_num(test[1])

20